In [1]:
!gdown 15Jhd2ZYh8fH8T79CslRwhTi9qqr1q041

Downloading...
From: https://drive.google.com/uc?id=15Jhd2ZYh8fH8T79CslRwhTi9qqr1q041
To: /content/best.pt
100% 14.4M/14.4M [00:00<00:00, 71.4MB/s]


In [2]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
%pip install -qr requirements.txt
%pip install -q roboflow

import torch
import os
from IPython.display import Image, clear_output

print(f"Versi torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

fatal: destination path 'yolov5' already exists and is not an empty directory.
/content/yolov5
Versi torch 1.12.1+cu113 (CPU)


In [3]:
from roboflow import Roboflow
rf = Roboflow(model_format="yolov5", notebook="ultralytics")

upload and label your dataset, and get an API KEY here: https://app.roboflow.com/?model=yolov5&ref=ultralytics


In [4]:
# buat folder
os.environ["DATASET_DIRECTORY"] = "/content/datasets"

In [5]:
#ambil dataset dari roboflow
from roboflow import Roboflow
rf = Roboflow(api_key="5s6pm49rP45QR1YXLlYL")
project = rf.workspace("daniel-morantha-yt377").project("ayamdeteksi")
dataset = project.version(2).download("yolov5")


# rf = Roboflow(api_key="5s6pm49rP45QR1YXLlYL")
# project = rf.workspace("daniel-morantha-yt377").project("ayamdeteksi")
# dataset = project.version(1).download("yolov5")

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to /content/datasets/AyamDeteksi-2 in yolov5pytorch:: 100%|██████████| 492/492 [00:00<00:00, 1109.11it/s]


**Prediksi untuk 1 Gambar Test**

In [6]:
# Model
model = torch.hub.load('ultralytics/yolov5', 'custom', '/content/best.pt')  # custom trained model

# Images
im = '/content/datasets/AyamDeteksi-2/test/images/20221104_081444_jpg.rf.6ce7a28d5c9cadc76eac0c426b0c76a7.jpg'  # or file, Path, URL, PIL, OpenCV, numpy, list

# Inference
results = model(im)

results.pandas().xyxy[0]

deteksi_class = results.pandas().xyxy[0].sort_values('confidence',ascending=False)['name'][0]
print(deteksi_class)

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2022-11-17 Python-3.7.15 torch-1.12.1+cu113 CPU

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


AyamSegar


**Prediksi untuk Semua File Gambar Test**

In [7]:
import glob
import os.path
file_test_images = glob.glob('/content/datasets/AyamDeteksi-2/test/images/*.jpg')

In [8]:
class_prediksi  = []
file_image_name = []

for file_image in file_test_images:
  image_name = os.path.basename(file_image).split('.')[0]
  results = model(file_image)
  results.pandas().xyxy[0]
  deteksi_class = results.pandas().xyxy[0].sort_values('confidence',ascending=False)['name'][0]

  class_prediksi.append(deteksi_class)
  file_image_name.append(image_name)

In [9]:
import pandas as pd
df_prediksi_test = pd.DataFrame({'image_name':file_image_name,
                                 'prediksi':class_prediksi})

In [10]:
df_prediksi_test

,image_name,prediksi
0,IMG_5029_jpg,AyamTiren
1,20221104_081444_jpg,AyamSegar
2,IMG_5234_JPG,AyamSegar
3,IMG_5038_jpg,AyamTiren
4,IMG_5075_jpg,AyamTiren
5,20221104_082508_jpg,AyamSegar
6,20221104_081504_jpg,AyamSegar
7,IMG_5068_jpg,AyamTiren
8,20221104_082538_jpg,AyamSegar
9,IMG_5032_jpg,AyamTiren


In [11]:
file_test_labels = glob.glob('/content/datasets/AyamDeteksi-2/test/labels/*.txt')

In [12]:
class_actual  = []
file_image_name = []

labels = ['AyamSegar','AyamTiren']

for file_label in file_test_labels:
  image_name = os.path.basename(file_label).split('.')[0]
  file_text = open(file_label, 'r')
  file_lines = file_text.readlines()
  class_actual.append(labels[int(file_lines[0].split(' ')[0])])
  file_image_name.append(image_name)

In [13]:
df_label_test = pd.DataFrame({'image_name':file_image_name,
                              'label':class_actual})

In [14]:
df_label_test

,image_name,label
0,IMG_5038_jpg,AyamTiren
1,20221104_082538_jpg,AyamSegar
2,IMG_5234_JPG,AyamSegar
3,20221104_081504_jpg,AyamSegar
4,20221104_082508_jpg,AyamSegar
5,IMG_5075_jpg,AyamTiren
6,20221104_081444_jpg,AyamSegar
7,IMG_5032_jpg,AyamTiren
8,IMG_5029_jpg,AyamTiren
9,IMG_5068_jpg,AyamTiren


In [15]:
df_prediksi_test = pd.merge(
    df_prediksi_test,
    df_label_test,
    how="inner",
    on='image_name')

In [16]:
df_prediksi_test

,image_name,prediksi,label
0,IMG_5029_jpg,AyamTiren,AyamTiren
1,20221104_081444_jpg,AyamSegar,AyamSegar
2,IMG_5234_JPG,AyamSegar,AyamSegar
3,IMG_5038_jpg,AyamTiren,AyamTiren
4,IMG_5075_jpg,AyamTiren,AyamTiren
5,20221104_082508_jpg,AyamSegar,AyamSegar
6,20221104_081504_jpg,AyamSegar,AyamSegar
7,IMG_5068_jpg,AyamTiren,AyamTiren
8,20221104_082538_jpg,AyamSegar,AyamSegar
9,IMG_5032_jpg,AyamTiren,AyamTiren


In [17]:
from sklearn.metrics import confusion_matrix
y_actual = df_prediksi_test['label']
y_pred = df_prediksi_test['prediksi']
confusion_matrix(y_actual, y_pred)

array([[5, 0],
       [0, 5]])

#Evaluasi Data val 

In [18]:
# Model
model = torch.hub.load('ultralytics/yolov5', 'custom', '/content/best.pt')  # custom trained model

# Images
im = '/content/datasets/AyamDeteksi-2/valid/images/20221104_081226_jpg.rf.425359a3b74e824c5ffe5182e3b5558f.jpg'  # or file, Path, URL, PIL, OpenCV, numpy, list

# Inference
results = model(im)

results.pandas().xyxy[0]

deteksi_class = results.pandas().xyxy[0].sort_values('confidence',ascending=False)['name'][0]
print(deteksi_class)

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2022-11-17 Python-3.7.15 torch-1.12.1+cu113 CPU

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


AyamSegar


In [19]:
import glob
import os.path
file_test_images = glob.glob('/content/datasets/AyamDeteksi-2/valid/images/*.jpg')

In [20]:
class_prediksi  = []
file_image_name = []

for file_image in file_test_images:
  image_name = os.path.basename(file_image).split('.')[0]
  results = model(file_image)
  results.pandas().xyxy[0]
  deteksi_class = results.pandas().xyxy[0].sort_values('confidence',ascending=False)['name'][0]

  class_prediksi.append(deteksi_class)
  file_image_name.append(image_name)

In [21]:
import pandas as pd
df_prediksi_val = pd.DataFrame({'image_name':file_image_name,
                                 'prediksi':class_prediksi})

In [22]:
df_prediksi_val

,image_name,prediksi
0,IMG_5067_jpg,AyamTiren
1,20221104_082515_jpg,AyamSegar
2,IMG_5122_jpg,AyamTiren
3,20221104_082532_jpg,AyamSegar
4,20221104_081233_jpg,AyamSegar
5,20221115_133221_jpg,AyamSegar
6,IMG_5238_JPG,AyamSegar
7,20221104_081241_jpg,AyamSegar
8,IMG_5035_jpg,AyamTiren
9,IMG_4924_jpg,AyamTiren


In [23]:
file_test_labels = glob.glob('/content/datasets/AyamDeteksi-2/valid/labels/*.txt')

In [24]:
class_actual  = []
file_image_name = []

labels = ['AyamSegar','AyamTiren']

for file_label in file_test_labels:
  image_name = os.path.basename(file_label).split('.')[0]
  file_text = open(file_label, 'r')
  file_lines = file_text.readlines()
  class_actual.append(labels[int(file_lines[0].split(' ')[0])])
  file_image_name.append(image_name)

In [25]:
df_label_val = pd.DataFrame({'image_name':file_image_name,
                              'label':class_actual})

In [26]:
df_label_val

,image_name,label
0,20221104_081217_jpg,AyamSegar
1,IMG_5066_jpg,AyamTiren
2,20221104_082512_jpg,AyamSegar
3,IMG_5035_jpg,AyamTiren
4,IMG_4924_jpg,AyamTiren
5,20221104_081233_jpg,AyamSegar
6,20221104_082515_jpg,AyamSegar
7,20221104_081228_jpg,AyamSegar
8,IMG_5067_jpg,AyamTiren
9,IMG_5040_jpg,AyamTiren


In [27]:
df_prediksi_val = pd.merge(
    df_prediksi_val,
    df_label_val,
    how="inner",
    on='image_name')

In [28]:
df_prediksi_val

,image_name,prediksi,label
0,IMG_5067_jpg,AyamTiren,AyamTiren
1,20221104_082515_jpg,AyamSegar,AyamSegar
2,IMG_5122_jpg,AyamTiren,AyamTiren
3,20221104_082532_jpg,AyamSegar,AyamSegar
4,20221104_081233_jpg,AyamSegar,AyamSegar
5,20221115_133221_jpg,AyamSegar,AyamSegar
6,IMG_5238_JPG,AyamSegar,AyamSegar
7,20221104_081241_jpg,AyamSegar,AyamSegar
8,IMG_5035_jpg,AyamTiren,AyamTiren
9,IMG_4924_jpg,AyamTiren,AyamTiren


In [39]:
from sklearn.metrics import confusion_matrix
y_actual = df_prediksi_val['label']
y_pred = df_prediksi_val['prediksi']
confusion_matrix(y_actual, y_pred)

array([[11,  0],
       [ 0,  9]])